In [15]:
import pickle
import numpy as np
from keras.models import Model
from keras.layers import Input, Lambda, Reshape, LSTM, Dense
from keras.optimizers import Adam
from keras import backend as K

In [7]:
tokenized_text, char_to_idx = pickle.load(open('../data/processed-alice.pickle', 'rb'))
X, Y = pickle.load(open('../data/alice-x-y-data.pickle', 'rb'))

Y = np.reshape(Y, (X.shape[1], X.shape[0], X.shape[2]))

# Global Variables
m, Tx, n_values = X.shape
n_a = 64 # for LSTM with 64-dimensional hidden states

# Train the Model
First we train the model to get the proper weights.
Remember, we must declare the layers in our model as global variables so they
can be reused in the Text Generation step

In [8]:
# Reused Layer for Text Generation: Reshape, LSTM, Dense
reshapor = Reshape((1, n_values))
LSTM_cell = LSTM(n_a, return_state=True)
densor = Dense(n_values, activation='softmax')

In [23]:
def learn_to_talk(Tx, n_a, n_values):
    X = Input(shape=(Tx, n_values))
    
    # Define initial hidden state for LSTM
    a0 = Input(shape=(n_a,), name='a0')
    c0 = Input(shape=(n_a,), name='c0')
    a = a0
    c = c0
    
    outputs = []
    for t in range(Tx):
        # Get t'th vector
        x = Lambda(lambda x: x[:,t,:])(X)
        # Reshape x to be (1,Tx)
        x = reshapor(x)
        # Perform 1 step of LSTM
        a, _, c = LSTM_cell(x)
        # apply densor to hidden state output of LSTM_cell
        out = densor(a)
        outputs.append(out)
    return Model([X, a0, c0], outputs)

In [24]:
model = learn_to_talk(Tx, n_a, n_values)

In [26]:
opt = Adam(lr=0.01, beta_1=0.9, beta_2=0.999, decay=0.01)
model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])

In [7]:
a0 = np.zeros((m, n_a))
c0 = np.zeros((m, n_a))

In [8]:
model.fit([X, a0, c0], list(Y), epochs=1)

Epoch 1/1
973/973 [==============================] - 162s 166ms/step - loss: 448.6111 - dense_1_loss: 3.4800 - dense_1_acc: 0.1624 - dense_1_acc_1: 0.3669 - dense_1_acc_2: 0.4286 - dense_1_acc_3: 0.4275 - dense_1_acc_4: 0.3823 - dense_1_acc_5: 0.1357 - dense_1_acc_6: 0.1316 - dense_1_acc_7: 0.0185 - dense_1_acc_8: 0.1768 - dense_1_acc_9: 0.2497 - dense_1_acc_10: 0.2425 - dense_1_acc_11: 0.3895 - dense_1_acc_12: 0.4306 - dense_1_acc_13: 0.4851 - dense_1_acc_14: 0.2878 - dense_1_acc_15: 0.2271 - dense_1_acc_16: 0.2559 - dense_1_acc_17: 0.2004 - dense_1_acc_18: 0.2765 - dense_1_acc_19: 0.1788 - dense_1_acc_20: 0.2528 - dense_1_acc_21: 0.3679 - dense_1_acc_22: 0.3443 - dense_1_acc_23: 0.0226 - dense_1_acc_24: 0.3083 - dense_1_acc_25: 0.2703 - dense_1_acc_26: 0.2117 - dense_1_acc_27: 0.1768 - dense_1_acc_28: 0.3813 - dense_1_acc_29: 0.4974 - dense_1_acc_30: 0.3145 - dense_1_acc_31: 0.3248 - dense_1_acc_32: 0.1387 - dense_1_acc_33: 0.2210 - dense_1_acc_34: 0.4481 - dense_1_acc_35: 0.2816 - d

# Use Trained Weights for Text Generation

In [28]:
model.save('test.h5')